In [40]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import Vantage
from pylabrobot.resources.hamilton import VantageDeck
backend = VantageDeck(size=1.3)
lh = LiquidHandler(backend=Vantage(), deck=backend)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
await lh.setup()

In [42]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L,  #""" Tip Rack with 96 1000ul High Volume Tip with filter """
    LT_L,   #""" Tip Rack with 96 10ul Low Volume Tip with filter """
    STF_L,  #""" Tip Rack with 96 300ul Standard Volume Tip with filter """
)

In [44]:
lh.summary()

Rail     Resource                   Type                Coordinates (mm)
(22) ├── trash                      Trash               (577.500, 185.600, 137.100)



In [45]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = LT_L(name='tips_10')
tip_car[1] = STF_L(name='tips_300')
tip_car[2] = HTF_L(name='tips_1000')

lh.deck.assign_child_resource(tip_car, rails=23)

plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='dye') #Load dye into the A1:H1
plt_car[1] = Cos_96_DW_1mL(name='plate_01')
plt_car[2] = Cos_96_DW_1mL(name='source')

lh.deck.assign_child_resource(plt_car, rails=17)

lh.summary()

Rail     Resource                   Type                Coordinates (mm)
(14) ├── plate carrier              PlateCarrier        (392.500, 063.000, 100.000)
     │   ├── dye                    Plate               (410.500, 146.000, 187.150)
     │   ├── plate_01               Plate               (410.500, 242.000, 187.150)
     │   ├── source                 Plate               (410.500, 338.000, 187.150)
     │   ├── <empty>
     │   ├── <empty>
     │
(20) ├── tip carrier                TipCarrier          (527.500, 063.000, 100.000)
     │   ├── tips_10                TipRack             (545.400, 145.800, 192.450)
     │   ├── tips_300               TipRack             (545.400, 241.800, 164.450)
     │   ├── tips_1000              TipRack             (545.400, 337.800, 131.450)
     │   ├── <empty>
     │   ├── <empty>
     │
(22) ├── trash                      Trash               (577.500, 185.600, 137.100)



In [46]:

def serial_dilution(stock_concentration, dilution_ratio, num_dilutions, initial_volume=300):

    diluent_volumes = []
    water_volumes = []
    current_volume = initial_volume
    for i in range(num_dilutions):
        diluent_volume = current_volume - (current_volume / dilution_ratio)
        water_volume = initial_volume - diluent_volume
        diluent_volumes.append(diluent_volume)
        water_volumes.append(water_volume)
        current_volume = current_volume / dilution_ratio
    return diluent_volumes, water_volumes

In [47]:
# Example usage:
stock_concentration = 100  # mg/mL
dilution_ratio = 2  # 1:10 dilution
num_dilutions = 6

diluent_volumes, water_volumes = serial_dilution(stock_concentration, dilution_ratio, num_dilutions)

print("Diluent Volumes (uL):")
print(diluent_volumes)
print("Water Volumes (uL):")
print(water_volumes)

Diluent Volumes (uL):
[150.0, 75.0, 37.5, 18.75, 9.375, 4.6875]
Water Volumes (uL):
[150.0, 225.0, 262.5, 281.25, 290.625, 295.3125]


In [63]:
tiprack_1 = lh.deck.get_resource("tips_10")
tiprack_2 = lh.deck.get_resource("tips_300")
tiprack_3 = lh.deck.get_resource("tips_1000")
await lh.pick_up_tips(tiprack_2["A1:H1"])

In [64]:
dye = lh.deck.get_resource("dye")
water = lh.deck.get_resource("source")
plate1 = lh.deck.get_resource("plate_01")



In [66]:
await lh.drop_tips(tiprack_2["A1:H1"])

In [67]:
await lh.pick_up_tips(tiprack_2["A2:H2"])

In [72]:
stock_concentration = 100  # mg/mL
dilution_ratio = 2  # 1:10 dilution
num_dilutions = 6

diluent_volumes, water_volumes = serial_dilution(stock_concentration, dilution_ratio, num_dilutions)
await lh.aspirate(water["A1:F1"], vols=water_volumes)
await lh.dispense(plate1["A1:F1"], vols=water_volumes)

In [73]:
await lh.drop_tips(tiprack_2["A2:H2"])